Training and forecasting the solar power of Hokkaido Island in Japna
Training 2017, Forecast 2018

https://stackoverflow.com/questions/35139108/how-to-install-xgboost-in-anaconda-python-windows-platform

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import csv 
import matplotlib.pyplot as plt
import pytz
import xgboost as xgb

In [2]:
import os
cwd = os.getcwd()
cwd
# os.chdir('pvinhokkaidodatadistribution')
# C:\Users\Mhdella\Google Drive\Jupyter Folder\PV_Hokkaido_forecast_contest\
os.chdir('C:\\Users\\Mhdella\\Google Drive\\Jupyter Folder\\PV_Hokkaido_forecast_contest\\pvinhokkaidodatadistribution')
nwd=os.getcwd()
nwd

'C:\\Users\\Mhdella\\Google Drive\\Jupyter Folder\\PV_Hokkaido_forecast_contest\\pvinhokkaidodatadistribution'

In [3]:
# file_name='Merg_Ens_Agg_42_44_mes17_18.csv'
# file_name='Merg_Ens_Agg_43_mes17_18.csv'

# file_name='Merg_Ens_Agg_42_44_mes16_17_18.csv'
file_name='Merg_Ens_mmaxmin_Agg_42_44_mes16_18.csv'

# file_name='Merg_Res_Comb_S1_S2_2018_tz_Jp.csv'
# file_mane='Merg_Res_Comb_S1_S2_2017_tz_Jp.csv'

df_hok=pd.read_csv(file_name) 
df_hok.head()
# df_hok.tail()

,DATE,TIME,S1[MW],S2[MW],solar[kW/m2],temp[deg C],solar[kW/m2].1,temp[deg C].1,solar[kW/m2].2,temp[deg C].2,...,t2m.2,asn.2,sd.2,ssrdc.2,ssrd.2,tcc.2,tciw.2,tclw.2,tp.2,wind_speed.2
0,1/1/2016,9:00:00,54.7,78.4,0.159228,-1.2,0.276627,0.0,0.339371,-5.9,...,-13.909399,0.600965,7.450000e-09,57.319398,30.413241,0.000000,0.000000,0.000000,0.000000e+00,7.704484
1,1/1/2016,9:30:00,113.8,89.0,0.221269,-0.3,0.307764,0.1,0.297246,-4.4,...,-12.694835,0.600870,7.450000e-09,104.414120,55.211142,0.037510,0.000396,0.000012,2.690000e-07,7.772519
2,1/1/2016,10:00:00,115.9,97.3,0.231619,-0.5,0.354492,0.7,0.320564,-2.3,...,-11.480271,0.600775,7.450000e-09,151.508843,80.009043,0.075021,0.000793,0.000023,5.390000e-07,7.840554
3,1/1/2016,10:30:00,139.5,100.6,0.205909,-0.4,0.363168,0.8,0.306980,-1.9,...,-10.265707,0.600680,7.450000e-09,198.603565,104.806944,0.112531,0.001189,0.000034,8.080000e-07,7.908589
4,1/1/2016,11:00:00,125.7,96.8,0.277444,-0.2,0.358989,0.6,0.237579,-3.1,...,-9.051143,0.600586,7.450000e-09,245.698287,129.604846,0.150041,0.001585,0.000046,1.080000e-06,7.976625


In [4]:
#### This cell just to check out the dataset with different coordination lat 43 and lat42-44 from ECMWF
df_hok1=pd.read_csv('Merg_Ens_Agg_42_44_mes17_18.csv') 
df_hok2=pd.read_csv('Merg_Ens_Agg_43_mes17_18.csv')

df_mns= {'col1': df_hok1.mean(), 'col2': df_hok2.mean()}
# df_mns

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-regression-py

In [5]:
import sklearn 
import numpy as np
from sklearn.metrics import mean_squared_error
# X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
# X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
df_hok.head()

,DATE,TIME,S1[MW],S2[MW],solar[kW/m2],temp[deg C],solar[kW/m2].1,temp[deg C].1,solar[kW/m2].2,temp[deg C].2,...,t2m.2,asn.2,sd.2,ssrdc.2,ssrd.2,tcc.2,tciw.2,tclw.2,tp.2,wind_speed.2
0,1/1/2016,9:00:00,54.7,78.4,0.159228,-1.2,0.276627,0.0,0.339371,-5.9,...,-13.909399,0.600965,7.450000e-09,57.319398,30.413241,0.000000,0.000000,0.000000,0.000000e+00,7.704484
1,1/1/2016,9:30:00,113.8,89.0,0.221269,-0.3,0.307764,0.1,0.297246,-4.4,...,-12.694835,0.600870,7.450000e-09,104.414120,55.211142,0.037510,0.000396,0.000012,2.690000e-07,7.772519
2,1/1/2016,10:00:00,115.9,97.3,0.231619,-0.5,0.354492,0.7,0.320564,-2.3,...,-11.480271,0.600775,7.450000e-09,151.508843,80.009043,0.075021,0.000793,0.000023,5.390000e-07,7.840554
3,1/1/2016,10:30:00,139.5,100.6,0.205909,-0.4,0.363168,0.8,0.306980,-1.9,...,-10.265707,0.600680,7.450000e-09,198.603565,104.806944,0.112531,0.001189,0.000034,8.080000e-07,7.908589
4,1/1/2016,11:00:00,125.7,96.8,0.277444,-0.2,0.358989,0.6,0.237579,-3.1,...,-9.051143,0.600586,7.450000e-09,245.698287,129.604846,0.150041,0.001585,0.000046,1.080000e-06,7.976625


In [6]:
print(list(df_hok))

['DATE', 'TIME', 'S1[MW]', 'S2[MW]', 'solar[kW/m2]', 'temp[deg C]', 'solar[kW/m2].1', 'temp[deg C].1', 'solar[kW/m2].2', 'temp[deg C].2', 'solar[kW/m2].3', 'temp[deg C].3', 'u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'ssrd', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'ssrd.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'ssrd.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']


In [7]:
#### df_feats=df_hok[df_hok['ssrdc'] != 0] ## dropping night hours, when clear-sky solar irradiance (SSRDC) forecasts=0, and this can be applied in real forecasts  

#### df_feats=df_hok.set_index('DATE')

############################# Not dropping night hours

df_tar_feats=df_hok.set_index(pd.to_datetime(df_hok['DATE'] + ' ' + df_hok['TIME']))

############## In case of forecast solar power  #######

Targ=df_tar_feats['S1[MW]']+df_tar_feats['S2[MW]']
Targ = Targ.replace(np.nan, 0, regex=True)

df_feats=df_tar_feats[['u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'ssrd', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 
             'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'ssrd.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 
             'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'ssrd.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']]
############### In case of forecast solar irradiance  #######

# Targ=df_tar_feats['ssrd']  #ssrd is the mean of ssrd data from all ensemble members
# df_feats=df_tar_feats[['u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 
#              'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 
#              'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']]


df_tar_feats.insert(loc=0, column='Targ', value=Targ)

df_feats.head()
df_feats.tail()

print(len(list(df_feats)), len(list(df_tar_feats)))
# Targ.tail()


36 49


In [8]:
len(df_feats)
# len(Targ)
print(len(Targ), max(Targ), Targ.mean())

52603 312.6 37.385689029143066


In [9]:
# ind_date_time= pd.date_range('2019-03-08 00:00','2019-03-09 00:00',freq='120T')
ind_date_time= pd.date_range('2019-03-08 04:00','2019-03-09 03:30',freq='30T')

Hours= ind_date_time.strftime('%H:%M')
# Hours
# for num in range(0,len(Hours),6):
#     print(Hours[num])

In [10]:
df_intrv0=df_tar_feats.between_time('20:00','03:30')
df_intrv1=df_tar_feats.between_time('04:00','04:30')
df_intrv2=df_tar_feats.between_time('05:00','05:30')
df_intrv3=df_tar_feats.between_time('06:00','06:30')
df_intrv4=df_tar_feats.between_time('07:00','07:30')
df_intrv5=df_tar_feats.between_time('08:00','08:30')
df_intrv6=df_tar_feats.between_time('09:00','09:30')
df_intrv7=df_tar_feats.between_time('10:00','10:30')
df_intrv8=df_tar_feats.between_time('11:00','11:30')
df_intrv9=df_tar_feats.between_time('12:00','12:30')
df_intrv10=df_tar_feats.between_time('13:00','13:30')
df_intrv11=df_tar_feats.between_time('14:00','14:30')
df_intrv12=df_tar_feats.between_time('15:00','15:30')
df_intrv13=df_tar_feats.between_time('16:00','16:30')
df_intrv14=df_tar_feats.between_time('17:00','17:30')
df_intrv15=df_tar_feats.between_time('18:00','18:30')
df_intrv16=df_tar_feats.between_time('19:00','19:30')
# df_intrv16.head(9)

In [11]:
#df=dfdfd #Just to stop before run the following cell

In [12]:
from Hokk_Hourly_Forecast_CV_Models import XGB_model_intrv_CV

intvs=[df_intrv0, df_intrv1, df_intrv2, df_intrv3, df_intrv4, df_intrv5, df_intrv6, df_intrv7, df_intrv8,
       df_intrv9, df_intrv10, df_intrv11, df_intrv12, df_intrv13, df_intrv14, df_intrv15, df_intrv16]

for num in range(0, len(intvs)):
    df_hr=intvs[num]
#     print(num, len(df_hr), max(df_hr['Targ']), df_hr['Targ'].mean(), df_hr['Targ'].std())
    
    ####Select features of the data 
    Targ=df_hr['Targ']
    df_feats_w_mes=df_hr.drop('Targ', axis=1) #Features including measurements and weather forecasts
    df_feats_no_mes =df_hr.drop(df_tar_feats.columns[range(0,13)], axis=1)  #Features of weather forecasts only

    # df_feats=df_feats_w_mes  ## the measurements lenght is less than the weather since there are not exist in forecast period
    df_feats=df_feats_no_mes   
#     X_train=df_feats.loc['1/1/2017':'12/31/2017']
#     X_test=df_feats.loc['1/1/2016':'12/31/2016']
    X_train=df_feats.loc['1/1/2016':'12/31/2016']
    X_test=df_feats.loc['1/1/2017':'12/31/2017']

    # X_train=df_feats.loc['1/1/2017':'12/31/2017']
    # X_test=df_feats.loc['1/1/2018':'12/31/2018']

#     Y_train=Targ.loc['1/1/2017':'12/31/2017']
#     Y_test=Targ.loc['1/1/2016':'12/31/2016']
    Y_train=Targ.loc['1/1/2016':'12/31/2016']
    Y_test=Targ.loc['1/1/2017':'12/31/2017']
    
    # Y_train=Targ.loc['1/1/2017':'12/31/2017']
    # Y_test=Targ.loc['1/1/2018':'12/31/2018']

    X_train.head()
    # X_train.tail()
    # len(X_train)

    # Y_train.head()
    # len(Y_train)
    Y_train.tail()
    
    if num==0:
        print('Yes20:00-3:30')
        XGB_model_intrv_CV(df_feats, Targ)    
    elif num==1:
        print('Yes04:00-04:30')
        XGB_model_intrv_CV(df_feats, Targ)
    elif num==2:
        print('Yes05:00-05:30')
        XGB_model_intrv_CV(df_feats, Targ)
    elif num==3:
        print('Yes06:00-06:30')
        XGB_model_intrv_CV(df_feats, Targ)
    elif num==4:
        print('Yes07:00-07:30')
        XGB_model_intrv_CV(df_feats, Targ)    
    elif num==5:
        print('Yes08:00-08:30')
        XGB_model_intrv_CV(df_feats, Targ)
    elif num==6:
        print('Yes09:00-09:30')
        XGB_model_intrv_CV(df_feats, Targ)
    elif num==7:
        print('Yes10:00-10:30')
        XGB_model_intrv_CV(df_feats, Targ)   
    elif num==8:
        print('Yes11:00-11:30')
        XGB_model_intrv_CV(df_feats, Targ) 
    elif num==9:
        print('Yes12:00-12:30')
        XGB_model_intrv_CV(df_feats, Targ) 
    elif num==10:
        print('Yes13:00-13:30')
        XGB_model_intrv_CV(df_feats, Targ) 
    elif num==11:
        print('Yes14:00-14:30')
        XGB_model_intrv_CV(df_feats, Targ)
    elif num==12:
        print('Yes15:00-15:30')
        XGB_model_intrv_CV(df_feats, Targ)    
    elif num==13:
        print('Yes16:00-16:30')
        XGB_model_intrv_CV(df_feats, Targ)  
    elif num==14:
        print('Yes17:00-17:30')
        XGB_model_intrv_CV(df_feats, Targ)
    elif num==15:
        print('Yes18:00-18:30')
        XGB_model_intrv_CV(df_feats, Targ)    
    elif num==16:
        print('Yes19:00-19:30')
        XGB_model_intrv_CV(df_feats, Targ)
    else:
        print('No')

Yes20:00-3:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   14.4s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   52.6s finished


{'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 100}
Yes04:00-04:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.7s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   27.6s finished


{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 200}
Yes05:00-05:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits


...

{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}
Yes18:00-18:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.7s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   29.6s finished


{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 100}
Yes19:00-19:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  40 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:    9.6s finished


{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 300}


In [13]:
MSE=mean_squared_error(df_ytgfst['Yts'], df_ytgfst['Ypd'])  
# NRMSE=np.sqrt(MSE)/df_ytgfst['Yts'].mean()  #mean of target solar power during 2017 only
NRMSE=np.sqrt(MSE)/df_tar_feats['Targ'].mean()  #mean of target solar power durning 2016 and 2017

print(MSE, NRMSE)

NameError: name 'df_ytgfst' is not defined

Run XGB CV models of 16 intervals for 2 foldes to forecast solar power, one fold for 2016 and another for 2017
The XGB CV search leads to:

Yes20:00-3:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   14.4s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   52.6s finished

{'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 100}
Yes04:00-04:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.7s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   27.6s finished

{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 200}
Yes05:00-05:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    8.4s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   37.1s finished

{'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 300}
Yes06:00-06:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    9.6s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   46.4s finished

{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200}
Yes07:00-07:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    9.5s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   45.8s finished

{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}
Yes08:00-08:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    8.5s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   44.7s finished

{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200}
Yes09:00-09:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    8.5s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   46.4s finished

{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 300}
Yes10:00-10:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    9.2s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   42.6s finished

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
Yes11:00-11:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   41.9s finished

{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 100}
Yes12:00-12:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    8.4s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   45.7s finished

{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 100}
Yes13:00-13:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    9.0s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   46.1s finished

{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}
Yes14:00-14:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    8.7s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   47.4s finished

{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}
Yes15:00-15:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    9.0s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   43.5s finished

{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}
Yes16:00-16:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    8.7s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   40.9s finished

{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}
Yes17:00-17:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    9.0s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   37.1s finished

{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}
Yes18:00-18:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.7s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:   29.6s finished

{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 100}
Yes19:00-19:30
Fitting 2 folds for each of 48 candidates, totalling 96 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  40 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:    9.6s finished

{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 300}


